In [1]:
%sql

CREATE TABLE STREAMING_DATA(
RECORD_ID INTEGER,
A_NUMBER DOUBLE,
A_TEXT STRING
)
LOCATION '/dbfs/mnt/test_data/streaming_db/streaming_data'

In [2]:
%sh

ls /dbfs/dbfs/mnt/test_data/streaming_db/streaming_data

In [3]:
%sql

INSERT INTO TABLE STREAMING_DATA VALUES (1, 1.0, 'first_record');

In [4]:
%sql

SELECT * FROM STREAMING_DATA

RECORD_ID,A_NUMBER,A_TEXT
1,1.0,first_record


In [5]:
%sh

ls /dbfs/dbfs/mnt/test_data/streaming_db/streaming_data

_committed_2894894926104193824
part-00000-tid-2894894926104193824-a22f6dea-6de0-454c-a7f2-f9035ad3c506-0-1-c000
_started_2894894926104193824
_SUCCESS

In [6]:
%sql

DESCRIBE EXTENDED STREAMING_DATA

col_name,data_type,comment
RECORD_ID,int,null
A_NUMBER,double,null
A_TEXT,string,null
,,
# Detailed Table Information,,
Database,default,
Table,streaming_data,
Owner,root,
Created Time,Thu Apr 23 20:18:47 UTC 2020,
Last Access,Thu Jan 01 00:00:00 UTC 1970,


In [7]:
from pyspark.sql.types import *

In [8]:
file_schema=StructType([StructField("record_str", StringType())])

In [9]:
file_stream_df = spark.readStream.option("sep", "\n").option("header", "false").schema(file_schema).csv("dbfs:/dbfs/mnt/test_data/streaming_db/streaming_data/*")

In [10]:
from pyspark.sql.functions import input_file_name,regexp_extract,col

def foreach_batch_function(df, epoch_id):
#   each dataframe will belong to a single 
  df_with_file_name = df.withColumn("filename", regexp_extract(input_file_name(), "^(.*[\/])", 1))
  df_with_file_name.cache()
  all_paths = df_with_file_name.select("filename").distinct().collect()
  
  for path in all_paths:
    directory=path.filename
    directory=directory.replace("dbfs:/dbfs/mnt/test_data", "dbfs:/dbfs/mnt/test_data_output")
    df_with_file_name.where(col("filename")==path.filename).drop("filename").write.csv(directory, mode="overwrite")
  
  

In [11]:
file_stream_df.writeStream.foreachBatch(foreach_batch_function).option("checkpointLocation", "/dbfs/mnt/checkpoint_location").trigger(once=True).start() 

Out[11]: <pyspark.sql.streaming.StreamingQuery at 0x7f74ab667fd0>

In [12]:
%sh

ls /dbfs/dbfs/mnt/test_data_output/streaming_db/streaming_data

_committed_418289343588507213
_committed_4972246487905498969
_committed_6774612304445160304
_committed_7283690281657878304
_committed_vacuum8849252174007722683
part-00000-tid-418289343588507213-bdc8a88e-2653-4cfe-8ce0-331f436cca91-806-1-c000.csv
_started_418289343588507213
_started_4972246487905498969
_started_6774612304445160304
_SUCCESS

In [13]:
display(spark.read.csv("dbfs:/dbfs/mnt/test_data_output/streaming_db/streaming_data"))

_c0
11.0first_record
